# Exercise 3: Advanced Cryptography (Homomorphic encryption)


This third exercise is on the topic of Advanced Cryptography (more specifically, Homomorphic Encryption). The learning goal of this exercise is twofold:
1. To learn about partial and full homomorphic encryption
2. To demonstrate potential usecases for both

## Partial Homomorphic Encryption (PHE)
We start this exercise by looking at Partial Homomorphic Encryption (PHE). For this, we'll look at the RSA and Paillier cryptosystems.

### RSA
Let's explore how RSA can be used for homomorphic encryption. More information on how the cryptosystem works can be found [here](https://en.wikipedia.org/wiki/RSA_(cryptosystem)#Operation).

First, choose two prime numbers $p$ and $q$:

In [1]:
p = 3
q = 11

Then compute modulo $N$ and coprime $T$:

In [2]:
N = p*q
T = (p-1)*(q-1)

Now, select a public key $e$ for which the following holds true:
- $2 < e < λ(N)$
- $gcd(e, λ(N)) = 1$
- $e$ and $λ(N)$ are coprime

Note that $λ(N)$ here refers to the [Carmichael function](https://en.wikipedia.org/wiki/Carmichael_function).

In [3]:
e = 3

We can now compute the private key $d$ where:
$$
    de ≡ 1 (mod λ(n))
$$

In [4]:
d = 7

This gives us the following keys:

In [5]:
publicKey = (e,N)
secretKey = (d, N)
print(f"PK = {publicKey}, SK = {secretKey}")

PK = (3, 33), SK = (7, 33)


Now that we have our keys, we can start encrypting! Suppose that we want to encrypt the string *CAR*, where each letter is represented by their place in the alphabet ($A=1$, $B=2$ etc.).

To compute the cyphertext $C$ of message $M$ (converted to an integer), we use:
$$
    C ≡ M^e(mod N)
$$

In [6]:
C= 3
A= 4
R= 18

cypher = [(C**e)%N, (A**e)%N, (R**e)%N]
print(cypher)

[27, 31, 24]


To decrypt cyphertext $C$, we use:
$$
    C^d ≡ (M^e)^d ≡ M (mod N)
$$

In [7]:
decrypted = [((cypherText**d)%N) for cypherText in cypher]
print(f"Decrypted cyphertext = {decrypted}, original message = {[C, A, R]}")

Decrypted cyphertext = [3, 4, 18], original message = [3, 4, 18]


#### Question 1
Let $c1=31$ and $c2=27$. Can you decrypt $c1$, $c2$ and $c=c1*c2$ by hand? 

In [8]:
# Your answer here
### Solution
c = 27*31

decrypted_c1 = 31**d%N
decrypted_c2 = 27**d%N
decrypted_c = c**d%N
decrypted_c1,decrypted_c2, decrypted_c
### Solution end

(4, 3, 12)

#### Question 2
To what operations does the homomorphic property apply to? How many iterations are supported?

*Your answer here*
<span style="color:red">
    Solution: It only applies to multiplication, with no limit to iterations
</span>

#### Question 3
Try to perform 100 multiplications on encrypted numbers. Is the result correct?

In [9]:
# Your code here
### Solution
import random
def Enc(m):
    return (m**e) % N

def Dec(c):
    return (c**d) % N

result = 1
enc_result = Enc(1)
for i in range(1, 101):
    rand = random.randint(1, 2)
    result *= rand
    enc_result *= Enc(rand)
    if result % N != Dec(enc_result):
        print(f"Invalid after {i} iterations")
        break
print("All done")
### Solution end   

All done


### Paillier
Let's consider a key pair `Pk` for the Paillier's scheme where
$$
    Pk = (n, g), Sk = (λ, μ)
$$
and
* $p = 13$
* $q = 17$
* $n = 221$
* $λ = 48$
* $g = 4886$
* $μ = 159$

In [10]:
p = 13
q = 17
n = 221
g = 4886
phi = 48
mu = 159

In [11]:
def encrypt(m, r):
    return ((g**m)*(r**n))%(n*n)

def decrypt(cyph):
    return ((((cyph ** phi) % (n * n) - 1) / n) * mu) % n

#### Question 4

Use the Paillier's scheme to do the following operations:
- Let $m1 = 58$ and $m2 = 126$.
- Choose two random numbers $r1$ and $r2$
- Encrypt $m1$ and $m2$ as $c1$ and $c2$ respectively
- Calculate $c = c1 * c2$
- Print the decrypted values of $c$, $c1$, $c2$

In [12]:
# Your code here
### Solution
m1 = 58
m2 = 126
r1 = 666
r2 = 999
c1 = encrypt(m1, r1)
c2 = encrypt(m2, r2)
c = c1 * c2
print(decrypt(c), decrypt(c1), decrypt(c2), 58 + 126)
### Solution end

184.0 58.0 126.0 184


#### Question 5
What can you note about $c$?
Does the homomorphic property apply to other types of operations as well?
How many iterations are supported?

*Your answer here*
<span style="color:red">
    Solution: It is only homomorphic over addition, unlimited iterations are supported.
</span>

#### Question 6
Why does this classify as Partial Homomorphic Encryption instead of Somewhat Homomorphic Encryption?

*Your answer here*
<span style="color:red">
    Solution: It classifies as PHE since it only supports one type of operation, but for an unlimited number of iterations.
</span>

## Somewhat Homomorphic Encryption (SWHE)


[Gentry's Blueprint](https://doi.org/10.1145/1666420.1666444) describes a general framework that transforms a SWHE scheme into a FHE scheme. In this task, we will work with the SWHE scheme that Gentry uses in his paper. While this is a symmetric encryption scheme, it can be transformed into an asymmetric scheme as well and can be used to encrypt a single bit.

In [13]:
LAMBDA = 8
DEBUG = False

import random, logging

def KeyGenSym(lam=LAMBDA):
    """
    Generate p,
    Random P-bit odd integer
    P = lam^2
    """
    while True:
        key = _generateBits(lam**2)
        if (key % 2 == 1):
            if DEBUG: print("p = {}".format(key))
            return key

def EncSym(p, m, lam=LAMBDA):
    """
    Encrypt m in {0, 1}
    """
    if m in [0, 1]:
        return generateM(m, lam) + p * generateQ(lam)
    raise ValueError("m must be in {0, 1}")

def DecSym(p, c):
    c_ = (c % p)
    if DEBUG: print("c' = {}".format(c_))
    return c_ % 2

def generateQ(lam):
    """
    Generate q,
    Random Q-bit integer
    Q = lam^5
    """
    return _generateBits(lam**5)

def generateM(m, lam):
    """
    Generate m',
    Random N-bit integer
    N = lam
    m' = m mod 2 
    """
    while True:
        M = _generateBits(lam)
        if (M - m) % 2 == 0:
            if DEBUG: print("m' = {}".format(M))
            return M

def _generateBits(length):
    while True:
        ran = random.getrandbits(length)
        if len(format(ran, 'b')) == length:
            return ran

#### Question 7
Generate a key $p$ and encrypt message $m = 1$ to ciphertext $c$. Then, decrypt $c$ and determine whether the decryption results in the correct message.

In [14]:
# Your code here
### Solution
p = KeyGenSym()
c = EncSym(p, 1)
print(DecSym(p, c))
### Solution end

1


#### Question 8
Encrypt two messages $m1$ and $m2$, then calculate the sum and product of these cypertexts, decrypt them and verify whether the results are correct.

In [15]:
# Your code here
### Solution
p = KeyGenSym()
m1 = 0
m2 = 1
c1 = EncSym(p, m1)
c2 = EncSym(p, m2)

c_sum = c1 + c2
c_product = c1 * c2

print(DecSym(p, c_sum))
print(DecSym(p, c_product))
### Solution end

1
0


#### Question 9
Demonstrate that noise will eventually prevent proper decryption by increasing the number of iterations of an operation. How many iterations can you perform before the decrypted result becomes invalid?

In [16]:
# Your code here
### Solution
count = 0
enc_count = 0
p = KeyGenSym()

for i in range(1, 1001):
    rand = random.randint(0,1)
    count += rand
    enc_count += EncSym(p, rand)
    if count - DecSym(p, enc_count) % 2 != 0:
        print("Invalid after {} iterations".format(i))
        break
### Solution end

Invalid after 2 iterations


## Full Homomorphic Encryption (FHE)
As mentioned in the lecture, this is considered somewhat of the "holy grail" of homomorphic encryption. Let's try it out and see how it differs from PHE.

For this exercise, we'll be making use of a package called [TenSEAL](https://github.com/OpenMined/TenSEAL) which provides a library for HE on vectors using [Microsoft SEAL](https://github.com/Microsoft/SEAL) (a HE library in C++). The first step is installing and importing TenSEAL into out notebook.

In [17]:
# Install the TenSEAL package
!pip install tenseal
import tenseal as ts

# Setup TenSEAL context
context = ts.context(ts.SCHEME_TYPE.BFV, poly_modulus_degree=4096, plain_modulus=1032193)

Great! Now we can start with actually doing some homomorphic encryption. Let's start of with an easy example:

In [18]:
# Example A
plain_vector = [100, 110, 120, 130, 140]
encrypted_vector = ts.bfv_vector(context, plain_vector)

add_result = encrypted_vector + [1, 2, 3, 4, 5]
print(add_result.decrypt())

substract_result = add_result - [10, 20, 30, 40, 50]
print(substract_result.decrypt())

multiplication_result = substract_result * [11, 12, 13, 14, 15]
print(multiplication_result.decrypt())

[101, 112, 123, 134, 145]
[91, 92, 93, 94, 95]
[1001, 1104, 1209, 1316, 1425]


As you may have noticed, we did not have to do any key management ourselves as the *TenSEALContext* is handling this for us. This also means that the *TenSEALContext* contains the secret key used for encryption! While this is handy for this exercise (as we can just tell it to decrypt without providing a key), it goes without saying that in real-world usecases, only the public key should be available to the party doing the processing (e.g. a cloud service).

#### Question 10
How does the example above differ from the earlier examples using PHE? I.e. how does this illustrate one of the major advantages of FHE compared to PHE?

*your answer here*
<span style="color:red">
Solution: This example shows that FHE supports multiple different types of operations on data. While not directly shown in the example, it also supports an unlimited number of iterations for these operations.
</span>

#### Question 11
Can you think of an example where PHE would not be suitable but FHE is?

*your answer here*
<span style="color:red">
    Solution: various answers can be correct, either mentioning a usecase where multiple different types of operations and/or multiple iterations are required.
</span>

Now, let's demonstrate a more realistic usecase of FHE in the following example. Here, we'll use FHE to encrypt data used for facial recognition. More specifically, we'll be using a library called [DeepFace](https://github.com/serengil/deepface).

We've already installed and initialized TenSEAL, so we're going to re-use the context we generated earlier. Again, please note that this context also contains the private key which you would normally not share with the party doing the processing.

However, to proceed, we will need to install and initialize DeepFace. We will also load some pictures of faces to use, which in this case will be of actor Nicolas Cage.
Note that the output below will throw some warnings stating the lack of cuda drivers on machines without a GPU (for example, when running on JupyterHub). DeepFace will still work, but will use (slower) CPU processing, which is fine for this exercise.

In [19]:
#[Client side]

# Install DeepFace
!pip install deepface
from deepface import DeepFace

# Generate new context
context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree = 8192, coeff_mod_bit_sizes = [60, 40, 40, 60])
context.generate_galois_keys()
context.global_scale = 2**40

# Define image paths
img1_path = "img/cage_1.jpg"
img2_path = "img/cage_2.jpg"

img1_embedding = DeepFace.represent(img1_path, model_name = 'VGG-Face')[0]["embedding"]
img2_embedding = DeepFace.represent(img2_path, model_name = 'VGG-Face')[0]["embedding"]

  Using cached deepface-0.0.79-py3-none-any.whl (49 kB)
  Using cached retina_face-0.0.13-py3-none-any.whl (16 kB)


2023-11-20 11:23:26.909883: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-20 11:23:26.949769: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 11:23:26.949824: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 11:23:26.950662: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 11:23:26.956941: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-20 11:23:26.957681: I tensorflow/core/platform/cpu_feature_guard.cc:1

We'll also define some functions to write and read the (encrypted) image data to and from files.

In [20]:
import base64

def write_data(file_name, data):
    if type(data) == bytes:
        #bytes to base64
        data = base64.b64encode(data)
         
    with open(file_name, 'wb') as f: 
        f.write(data)
 
def read_data(file_name):
    with open(file_name, "rb") as f:
        data = f.read()
    #base64 to bytes
    return base64.b64decode(data)

Okay, now we're all set to do some encryption!

In [21]:
#[Client side]
enc_v1 = ts.ckks_vector(context, img1_embedding)
enc_v2 = ts.ckks_vector(context, img2_embedding)
 
enc_v1_proto = enc_v1.serialize()
enc_v2_proto = enc_v2.serialize()
 
write_data("enc_v1.txt", enc_v1_proto)
write_data("enc_v2.txt", enc_v2_proto)

If all goes well, then you should now have two `.txt` files in the same directory as this notebook. These files contain encrypted data from the images used for facial recognition. Let's assume this encrypted data is sent to a server for facial recognition. What follows is code that runs on the server.

I will not go into detail on how this code works, you can find out more about that in both the source for this exercise and [here](https://sefiks.com/2018/08/13/cosine-similarity-in-machine-learning/) if you are interested.

In [22]:
#[Server side]
#restore the embedding of person 1
enc_v1_proto = read_data("enc_v1.txt")
enc_v1 = ts.lazy_ckks_vector_from(enc_v1_proto)
enc_v1.link_context(context)
 
#restore the embedding of person 2
enc_v2_proto = read_data("enc_v2.txt")
enc_v2 = ts.lazy_ckks_vector_from(enc_v2_proto)
enc_v2.link_context(context)
 
#euclidean distance
euclidean_squared = enc_v1 - enc_v2
euclidean_squared = euclidean_squared.dot(euclidean_squared)
 
#store the homomorphic encrypted squared euclidean distance
write_data("euclidean_squared.txt", euclidean_squared.serialize())

Alright, now let's get back to the client side and look at those results:

In [23]:
#load euclidean squared value
euclidean_squared_proto = read_data("euclidean_squared.txt")
euclidean_squared = ts.lazy_ckks_vector_from(euclidean_squared_proto)
euclidean_squared.link_context(context)
 
#decrypt it
euclidean_squared_plain = euclidean_squared.decrypt()[0]

if euclidean_squared_plain < 100:
    print("they are same person")
else:
    print("they are different persons")

they are same person


If all goes well then you should see that both images are recognized as the same person!

#### Question 12
Take a look at the `[Server side]` code of the example above, what information is actually shared with the cloud service?

*your answer here*
<span style="color: red">
    Solution: only encrypted data that Deepface extracted from the image are shared in this example. The image itself never leaves the client device!
</span>

### Acknowledgements
This exercise is adapted from the following:

PHE/SWHE:
- Student Takeover Exercise 2022

FHE:
- [Sefik Ilkin Serengil, A Step by Step Fully Homomorphic Encryption Example with TenSEAL in Python](https://sefiks.com/2023/04/10/a-step-by-step-fully-homomorphic-encryption-example-with-tenseal-in-python/)
- [Sefik Ilkin Serengil, Homomorphic Facial Recognition with TenSEAL](https://sefiks.com/2021/12/01/homomorphic-facial-recognition-with-tenseal/)
- [The TenSEAL Authors, TenSEAL Tutorials](https://github.com/OpenMined/TenSEAL/blob/main/tutorials/Tutorial%200%20-%20Getting%20Started.ipynb)

Images used:
- `cage_1.jpg` -> [CC-BY-SA Nicolas Genin](https://commons.wikimedia.org/wiki/File:Nicolas_Cage_-_66%C3%A8me_Festival_de_Venise_(Mostra).jpg)
- `cage_2.jpg` -> [CC-BY-SA Gerald Geronimo / G155 Multimedia](https://commons.wikimedia.org/wiki/File:Nicolas_Cage_Comic-Con_2011.jpg)